In [5]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import similarities

In [4]:
# Load the movielens-100k dataset (download it if needed).
data = pd.read_csv('./src/data/grailed.csv')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

ValueError: line_format parameter is incorrect.